In [3]:
import pandas as pd
import csv
import re

In [4]:
# names of files to read from
r_maxo_classes_with_definitionsTSV = '~/Git/MAxO/src/ontology/sparql-test/maxo_classes_with_definitions.tsv'
r_ncit_definitionsTSV = '~/Git/MAxO/src/ontology/sparql-test/ncit_definitions.tsv'

tsv_read_maxo = pd.read_csv(r_maxo_classes_with_definitionsTSV, sep='\t')
tsv_read_ncit = pd.read_csv(r_ncit_definitionsTSV, sep='\t')

maxo_id=list()
ncit_id=list()


                                            ?cls          ?xref  \
0  <http://purl.obolibrary.org/obo/MAXO_0000468>   NCIT:C159340   
1  <http://purl.obolibrary.org/obo/MAXO_0000066>    NCIT:C94624   
2  <http://purl.obolibrary.org/obo/MAXO_0000004>    NCIT:C15329   
3  <http://purl.obolibrary.org/obo/MAXO_0000746>  PMID:25006279   
4  <http://purl.obolibrary.org/obo/MAXO_0001006>    NCIT:C28133   

                                                ?def  
0  A minimally invasive procedure that permits sa...  
1  The administration of oxygen to an individual,...  
2  A diagnostic or treatment procedure performed ...  
3  A test to detect chromosomal breakage or aberr...  
4  The determination of the number of red blood c...  


In [4]:
tsv_read_maxo.columns

Index(['?cls', '?xref', '?def'], dtype='object')

In [5]:
from pandas import DataFrame

x=()
mylist=[]
newlist=list()
#extract just the maxo_id
maxo_id = pd.DataFrame(tsv_read_maxo)
cols = ["?cls","?xref","?def"]
maxo_id = maxo_id[maxo_id.columns[0]]
for line in maxo_id:
    line=line.strip('/')
    x=re.findall('[A-Z]{4,11}_[A-Z0-9]{1,15}', line)
    x=[item.replace('_', ':') for item in x]
    mylist.append(x)
    maxo_df= DataFrame(mylist,columns=['Maxo_ID'])

maxo_id_def= maxo_df.join(tsv_read_maxo, lsuffix="_left", rsuffix="_right")
print(maxo_id_def.head(2))
    
with open("maxo_xref_definitions.tsv",'wb') as out:
    maxo_id_def.to_csv('maxo_xref_definitions.tsv', encoding='utf-8', sep='\t', index=False)
    
    

        Maxo_ID                                           ?cls         ?xref  \
0  MAXO:0000468  <http://purl.obolibrary.org/obo/MAXO_0000468>  NCIT:C159340   
1  MAXO:0000066  <http://purl.obolibrary.org/obo/MAXO_0000066>   NCIT:C94624   

                                                ?def  
0  A minimally invasive procedure that permits sa...  
1  The administration of oxygen to an individual,...                                               ?cls  \
0  <http://purl.obolibrary.org/obo/NCIT_C158053>   
1   <http://purl.obolibrary.org/obo/NCIT_C81003>   

                                                ?def  
0  A change in the amino acid residue at position...  
1  The act of bending one's back forward from the...  


In [6]:
y=()
newlist=[]
#extract just the ncit_id
ncit_id = pd.DataFrame(tsv_read_ncit)
cols = ["?cls","?def"]
ncit_id = ncit_id[ncit_id.columns[0]]

for line in ncit_id:
    line=line.strip('/')
    y=re.findall('[A-Z]{4,11}_[A-Z0-9]{1,15}', line)
    y=[item.replace('_', ':') for item in y]
    newlist.append(y)
    ncit_df= DataFrame(newlist,columns=['NCIT_ID'])
ncit_id_def= ncit_df.join(tsv_read_ncit, lsuffix="_left", rsuffix="_right")
print(ncit_id_def.head(2))
    
with open("ncit_definitions.tsv",'wb') as out:
    ncit_id_def.to_csv('ncit_definitions.tsv', encoding='utf-8', sep='\t', index=False)
    

        NCIT_ID                                           ?cls  \
0  NCIT:C158053  <http://purl.obolibrary.org/obo/NCIT_C158053>   
1   NCIT:C81003   <http://purl.obolibrary.org/obo/NCIT_C81003>   

                                                ?def  
0  A change in the amino acid residue at position...  
1  The act of bending one's back forward from the...                                               ?cls  \
0  <http://purl.obolibrary.org/obo/NCIT_C158053>   
1   <http://purl.obolibrary.org/obo/NCIT_C81003>   

                                                ?def  
0  A change in the amino acid residue at position...  
1  The act of bending one's back forward from the...  


ncit_id_def.info

In [7]:

maxo_id_def.columns = ["Maxo_ID","?cls","ID","?def"]
print(maxo_id_def.head())

        Maxo_ID                                           ?cls             ID  \
0  MAXO:0000468  <http://purl.obolibrary.org/obo/MAXO_0000468>   NCIT:C159340   
1  MAXO:0000066  <http://purl.obolibrary.org/obo/MAXO_0000066>    NCIT:C94624   
2  MAXO:0000004  <http://purl.obolibrary.org/obo/MAXO_0000004>    NCIT:C15329   
3  MAXO:0000746  <http://purl.obolibrary.org/obo/MAXO_0000746>  PMID:25006279   
4  MAXO:0001006  <http://purl.obolibrary.org/obo/MAXO_0001006>    NCIT:C28133   

                                                ?def  
0  A minimally invasive procedure that permits sa...  
1  The administration of oxygen to an individual,...  
2  A diagnostic or treatment procedure performed ...  
3  A test to detect chromosomal breakage or aberr...  
4  The determination of the number of red blood c...  


In [21]:
maxo_id_list= []
maxo_def_list= []
maxo_def_xref_list= []
ncit_id_list=[]
ncit_def_list= []
for index, row in maxo_id_def.iterrows():
    if row[2].startswith("NCIT:"):
        for index, rows in ncit_id_def.iterrows():
            #determine if the the MAXO def xref matches the NCIT ID
            if row[2] == rows[0]:
                maxo_id_list.append(row[0])
                maxo_def_list.append(row[3])
                maxo_def_xref_list.append(row[2])
                ncit_id_list.append(rows[0])
                ncit_def_list.append(rows[2])
            else:
                continue


In [23]:

maxo_ncit_def_df=pd.DataFrame(list(zip(maxo_id_list, maxo_def_list, maxo_def_xref_list, ncit_id_list, ncit_def_list)), columns=["maxo_id","maxo_def", "maxo_def_xref","ncit_id", "ncit_def"])
print(maxo_ncit_def_df.head())

with open("maxo_ncit_def.tsv",'wb') as out:
    maxo_ncit_def_df.to_csv('maxo_ncit_def.tsv', encoding='utf-8', sep='\t', index=False)
    


        maxo_id                                           maxo_def  \
0  MAXO:0000468  A minimally invasive procedure that permits sa...   
1  MAXO:0000066  The administration of oxygen to an individual,...   
2  MAXO:0000004  A diagnostic or treatment procedure performed ...   
3  MAXO:0001006  The determination of the number of red blood c...   
4  MAXO:0000011  The treatment of deformity or disease through ...   

  maxo_def_xref       ncit_id  \
0  NCIT:C159340  NCIT:C159340   
1   NCIT:C94624   NCIT:C94624   
2   NCIT:C15329   NCIT:C15329   
3   NCIT:C28133   NCIT:C28133   
4   NCIT:C15302   NCIT:C15302   

                                            ncit_def  
0  A minimally invasive procedure that permits sa...  
1  The administration of oxygen to an individual,...  
2  A diagnostic or treatment procedure performed ...  
3  The determination of the number of red blood c...  
4  The treatment of deformity or disease through ...  
